Librerias
---

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Dataset
---

In [2]:
# Carga el dataset de películas
df = pd.read_parquet('Data/dataset_movies_to_ML.parquet')

Entendiendo y preparando los datos
---

In [3]:
df.columns

Index(['genres', 'overview', 'popularity', 'production_companies',
       'production_countries', 'title', 'vote_average', 'vote_count',
       'release_year', 'director', 'actor_names'],
      dtype='object')

In [4]:
len(df)

44513

In [5]:
df.head(5)

,genres,overview,popularity,production_companies,production_countries,title,vote_average,vote_count,release_year,director,actor_names
0,"[Animation, Comedy, Family]","Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],Toy Story,7.7,5415,1995,[John Lasseter],"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,"[Adventure, Fantasy, Family]",When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],Jumanji,6.9,2413,1995,[Joe Johnston],"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"[Romance, Comedy]",A family wedding reignites the ancient feud be...,11.712900,"[Warner Bros., Lancaster Gate]",[United States of America],Grumpier Old Men,6.5,92,1995,[Howard Deutch],"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,"[Comedy, Drama, Romance]","Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],Waiting to Exhale,6.1,34,1995,[Forest Whitaker],"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,[Comedy],Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],Father of the Bride Part II,5.7,173,1995,[Charles Shyer],"[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [40]:
indices = df[df['overview'] == ''].index

In [41]:
df = df.drop(index=indices)

In [43]:
df[df['overview'] == '']

,genres,overview,popularity,production_companies,production_countries,title,vote_average,vote_count,release_year,director,actor_names


In [44]:
df[df['overview'] == ""]

,genres,overview,popularity,production_companies,production_countries,title,vote_average,vote_count,release_year,director,actor_names


In [17]:
df['overview'] = df['overview'].str.strip()

In [36]:
df = df.dropna()

In [32]:
import re

In [34]:
df['overview'] = df['overview'].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))

In [45]:
df.info

<bound method DataFrame.info of                              genres  \
0       [Animation, Comedy, Family]   
1      [Adventure, Fantasy, Family]   
2                 [Romance, Comedy]   
3          [Comedy, Drama, Romance]   
4                          [Comedy]   
...                             ...   
45449      [Drama, Action, Romance]   
45450                       [Drama]   
45451     [Action, Drama, Thriller]   
45452                            []   
45453                            []   

                                                overview  popularity  \
0      Led by Woody Andys toys live happily in his ro...   21.946943   
1      When siblings Judy and Peter discover an encha...   17.015539   
2      A family wedding reignites the ancient feud be...   11.712900   
3      Cheated on mistreated and stepped on the women...    3.859495   
4      Just when George Banks has recovered from his ...    8.387519   
...                                                  ...         ..

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44513 entries, 0 to 45453
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                44513 non-null  object 
 1   overview              44513 non-null  object 
 2   popularity            44513 non-null  float64
 3   production_companies  44513 non-null  object 
 4   production_countries  44513 non-null  object 
 5   title                 44513 non-null  object 
 6   vote_average          44513 non-null  float64
 7   vote_count            44513 non-null  int64  
 8   release_year          44513 non-null  int32  
 9   director              44513 non-null  object 
 10  actor_names           44513 non-null  object 
dtypes: float64(2), int32(1), int64(1), object(7)
memory usage: 3.9+ MB


In [7]:
df['genres'].head(5)

0     [Animation, Comedy, Family]
1    [Adventure, Fantasy, Family]
2               [Romance, Comedy]
3        [Comedy, Drama, Romance]
4                        [Comedy]
Name: genres, dtype: object

In [8]:
df['overview'].head()

0    Led by Woody, Andy's toys live happily in his ...
1    When siblings Judy and Peter discover an encha...
2    A family wedding reignites the ancient feud be...
3    Cheated on, mistreated and stepped on, the wom...
4    Just when George Banks has recovered from his ...
Name: overview, dtype: object

In [9]:
df['overview'][0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [10]:
df['genres'].info()

<class 'pandas.core.series.Series'>
Index: 44513 entries, 0 to 45453
Series name: genres
Non-Null Count  Dtype 
--------------  ----- 
44513 non-null  object
dtypes: object(1)
memory usage: 1.7+ MB


In [11]:
df['actor_names'][0]

array(['Tom Hanks', 'Tim Allen', 'Don Rickles', 'Jim Varney',
       'Wallace Shawn', 'John Ratzenberger', 'Annie Potts', 'John Morris',
       'Erik von Detten', 'Laurie Metcalf', 'R. Lee Ermey',
       'Sarah Freeman', 'Penn Jillette'], dtype=object)

Las columnas produccion_companies, production_companies, 'production_countries', 'director' y 'actor_names' tienen datos estructurados en formato array de numpy: ej. [Animation, Comedy, Family]

In [6]:
# Convierte arrays de numpy a cadenas de texto
df['genres'] = df['genres'].apply(lambda x: ' '.join(x))

In [13]:
# Datos cambiados a texto, separado cada genero por espacio
df['genres']

0         Animation Comedy Family
1        Adventure Fantasy Family
2                  Romance Comedy
3            Comedy Drama Romance
4                          Comedy
                   ...           
45449        Drama Action Romance
45450                       Drama
45451       Action Drama Thriller
45452                            
45453                            
Name: genres, Length: 44513, dtype: object

In [14]:
df['genres'][0]

'Animation Comedy Family'

In [4]:
# Se convierten las columnas produccion_companies, production_companies, 'production_countries', 'director' y 'actor_names' en texto
df['production_companies'] = df['production_companies'].apply(lambda x: ' '.join(x))
df['production_countries'] = df['production_countries'].apply(lambda x: ' '.join(x))
df['director'] = df['director'].apply(lambda x: ' '.join(x))
df['actor_names'] = df['actor_names'].apply(lambda x: ' '.join(x))

In [16]:
df['actor_names']

0        Tom Hanks Tim Allen Don Rickles Jim Varney Wal...
1        Robin Williams Jonathan Hyde Kirsten Dunst Bra...
2        Walter Matthau Jack Lemmon Ann-Margret Sophia ...
3        Whitney Houston Angela Bassett Loretta Devine ...
4        Steve Martin Diane Keaton Martin Short Kimberl...
                               ...                        
45449    Patrick Bergin Uma Thurman David Morrissey Jür...
45450    Angel Aquino Perry Dizon Hazel Orencio Joel To...
45451    Erika Eleniak Adam Baldwin Julie du Page James...
45452    Iwan Mosschuchin Nathalie Lissenko Pavel Pavlo...
45453                                                     
Name: actor_names, Length: 44513, dtype: object

---
Modelo de recomendación por Generos
---

Vectorizacion

In [7]:
# Vectorizando y creando la matrix de cada genero
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['genres'])

Calculando similitud del coseno

In [8]:
# Calcula sla similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [9]:
# Crear un índice para mapear los títulos a los índices, y nos aseguramos que no existan duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Robin Hood                     45449
Century of Birthing            45450
Betrayal                       45451
Satan Triumphant               45452
Queerama                       45453
Length: 44513, dtype: int64

In [10]:
# Función que realiza la recomendación
def recomendar_peliculas_M1(dato, cosine_sim = cosine_sim):
    if df[df['title'] == dato].empty:
        retorno = 'La pelicula no existe'
    
    else:
        if (type(dato) == str):
            # Obtengo la posicion
            title = dato
            posicion = df[df['title'] == dato].index[0]
        elif (type(dato) == int):
            # Obtenemos el nombre de la pelicula
            title = df['title'][dato]
            posicion = dato

        idx = indices[posicion]   
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]  # Recomendamos las 5 películas más similares
        movie_indices = [i[0] for i in sim_scores]
        retorno = (f'Pelicula seleccionada: {title}, id: {posicion}\n\nPensamos que te puede interesar:\n{df['title'].iloc[movie_indices]}')
    
    return retorno 

# Prueba del modelo de recomendacion 
print(recomendar_peliculas_M1('fghfgh'))

La pelicula no existe


In [11]:
# Prueba del modelo de recomendacion 
M1_test01 = recomendar_peliculas_M1('Titanic')
print(M1_test01)

Pelicula seleccionada: Titanic, id: 1641

Pensamos que te puede interesar:
1697               Welcome to Woop Woop
34005             Jem and the Holograms
6322                            Jubilee
9453     The Saddest Music in the World
11924                      O Lucky Man!
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1322832937.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [12]:
# Prueba del modelo de recomendacion 
M1_test02 = recomendar_peliculas_M1('Batman')
print(M1_test02)

Pelicula seleccionada: Batman, id: 585

Pensamos que te puede interesar:
997     The Sword in the Stone
1001                     Dumbo
1612                 Anastasia
1973        Lady and the Tramp
1974        The Little Mermaid
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1322832937.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [13]:
# Prueba del modelo de recomendacion 
M1_test03 = recomendar_peliculas_M1('Sabrina')
print(M1_test03)

Pelicula seleccionada: Sabrina, id: 6

Pensamos que te puede interesar:
6               Sabrina
51     Mighty Aphrodite
63        Two If by Sea
67         French Twist
100           Mr. Wrong
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1322832937.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [14]:
# Prueba del modelo de recomendacion 
M1_test04 = recomendar_peliculas_M1('Toy Story')
print(M1_test04)

Pelicula seleccionada: Toy Story, id: 0

Pensamos que te puede interesar:
696              Oliver & Company
727                 A Close Shave
1111           The Wrong Trousers
1720             Meet the Deedles
1941    The Great Mouse Detective
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1322832937.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [15]:
# Prueba del modelo de recomendacion 
M1_test05 = recomendar_peliculas_M1('Pinocchio')
print(M1_test05)

Pelicula seleccionada: Pinocchio, id: 589

Pensamos que te puede interesar:
315      Shallow Grave
330     The Underneath
476         Kalifornia
593     Love and a .45
1058    Reservoir Dogs
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1322832937.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [68]:
# Cuantas veces se encuentra Titanic en el dataset
tit = df[df['title'] == 'Titanic']
print(tit['overview'][1641])
print(tit['overview'][3288])
print(tit['overview'][1641])

84 years later, a 101-year-old woman named Rose DeWitt Bukater tells the story to her granddaughter Lizzy Calvert, Brock Lovett, Lewis Bodine, Bobby Buell and Anatoly Mikailavich on the Keldysh about her life set in April 10th 1912, on a ship called Titanic when young Rose boards the departing ship with the upper-class passengers and her mother, Ruth DeWitt Bukater, and her fiancé, Caledon Hockley. Meanwhile, a drifter and artist named Jack Dawson and his best friend Fabrizio De Rossi win third-class tickets to the ship in a game. And she explains the whole story from departure until the death of Titanic on its first and last voyage April 15th, 1912 at 2:20 in the morning.
Unhappily married, Julia Sturges decides to go to America with her two children on the unsinkable Titanic. Her husband, Richard also arranges passage on the luxury liner so as to have custody of their two children. All this fades to insignificance once the ship hits an iceberg.
84 years later, a 101-year-old woman na

In [24]:
indices['Batman']

title
Batman     585
Batman    8611
dtype: int64

In [25]:
indices['Sabrina']

title
Sabrina      6
Sabrina    888
dtype: int64

In [26]:
indices['Titanic']

title
Titanic     1641
Titanic     3288
Titanic    25492
dtype: int64

El sistema debe ser capaz de recomendar por contenido cuando es elegida una pelicula, la finalidad es recomendar otra de caracteristicas similares como los casos que se muestran aquí, dado que no recomienda estas películas, el modelo no es funcional en su totalidad, tambien se observa que no exste variedad en la recomendación puesto que sólo retorna películas de indices pequeños, para mejorarlo, se agregan caracteríssticas al modelo, una de ellas es overview.

---
Modelo de recomendación con las características overview y género
---

In [16]:
df['combined_features'] = df['genres'] + ' ' + df['overview']

In [17]:
df['combined_features'][0]

"Animation Comedy Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences."

In [18]:
# Vectorización
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

In [19]:
# Calcular similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [20]:
# Crear un índice para mapear los títulos a los índices, y nos aseguramos que no existan duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Robin Hood                     45449
Century of Birthing            45450
Betrayal                       45451
Satan Triumphant               45452
Queerama                       45453
Length: 44513, dtype: int64

In [21]:
# Función que realiza la recomendación
def recomendar_peliculas_M2(dato, cosine_sim = cosine_sim):
    if df[df['title'] == dato].empty:
        retorno = 'La pelicula no existe'
    
    else:
        if (type(dato) == str):
            # Obtengo la posicion
            title = dato
            posicion = df[df['title'] == dato].index[0]
        elif (type(dato) == int):
            # Obtenemos el nombre de la pelicula
            title = df['title'][dato]
            posicion = dato

        idx = indices[posicion]   
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]  # Recomendamos las 5 películas más similares
        movie_indices = [i[0] for i in sim_scores]
        retorno = (f'Pelicula seleccionada: {title}, id: {posicion}\n\nPensamos que te puede interesar:\n{df['title'].iloc[movie_indices]}')
    
    return retorno 

# Prueba del modelo de recomendacion 
print(recomendar_peliculas_M2('fghfgh'))

La pelicula no existe


In [27]:
# Prueba del modelo de recomendacion 
M2_test01 = recomendar_peliculas_M2('Titanic')
print(M2_test01)

Pelicula seleccionada: Titanic, id: 1641

Pensamos que te puede interesar:
10247             A Touch of Spice
26091    Swinging with the Finkels
13251                  Girls Rock!
29486    Girltrash: All Night Long
1914                          Dune
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\2174510726.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [28]:
# Prueba del modelo de recomendacion 
M2_test02 = recomendar_peliculas_M2('Batman')
print(M2_test02)

Pelicula seleccionada: Batman, id: 585

Pensamos que te puede interesar:
36495          Pinocchio's Christmas
803      The Adventures of Pinocchio
27719                      Pinocchio
17967                      Pinocchio
30831            Pinocchio's Revenge
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\2174510726.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [29]:
# Prueba del modelo de recomendacion 
M2_test03 = recomendar_peliculas_M2('Sabrina')
print(M2_test03)

Pelicula seleccionada: Sabrina, id: 6

Pensamos que te puede interesar:
44857                         Ugly Melanie
11595            The Ugly Duckling and Me!
4101                          The Brothers
28426                  How to Fall in Love
35578    Bakit Hindi Ka Crush Ng Crush Mo?
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\2174510726.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [30]:
# Prueba del modelo de recomendacion 
M2_test04 = recomendar_peliculas_M2('Toy Story')
print(M2_test04)

Pelicula seleccionada: Toy Story, id: 0

Pensamos que te puede interesar:
15371                    Toy Story 3
3000                     Toy Story 2
24542                      Small Fry
10312         The 40 Year Old Virgin
23863    Andy Hardy's Blonde Trouble
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\2174510726.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [31]:
# Prueba del modelo de recomendacion 
M2_test05 = recomendar_peliculas_M2('Pinocchio')
print(M2_test05)

Pelicula seleccionada: Pinocchio, id: 589

Pensamos que te puede interesar:
1740              Nil by Mouth
13844      Breakfast with Scot
8969         Across the Tracks
415      Beverly Hills Cop III
37972                     Joey
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\2174510726.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


El sistema de recomendación mejoró puesto que en algunos casos hace la recomendación acorde a peliculas similares como el caso particular de Toy Story, en donde recomienda películas de la saga, en contraste al buscar el titulo de la pelicula Batman, la recomendación no funciona bien mismo caso con Pinocchio. Se agregará una característica más para mejorar el modelo.

---
Modelo de recomendación con las características overview, género y production_companies
---

In [32]:
df['combined_features'] = df['genres'] + ' ' + df['overview'] + ' ' + df['production_companies']

In [33]:
# Agrega al final Pixar studios
df['combined_features'][0]

"Animation Comedy Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. Pixar Animation Studios"

In [34]:
# Vectorización
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

In [35]:
# Calcular similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [36]:
# Crear un índice para mapear los títulos a los índices, y nos aseguramos que no existan duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Robin Hood                     45449
Century of Birthing            45450
Betrayal                       45451
Satan Triumphant               45452
Queerama                       45453
Length: 44513, dtype: int64

In [37]:
# Función que realiza la recomendación
def recomendar_peliculas_M3(dato, cosine_sim = cosine_sim):
    if df[df['title'] == dato].empty:
        retorno = 'La pelicula no existe'
    
    else:
        if (type(dato) == str):
            # Obtengo la posicion
            title = dato
            posicion = df[df['title'] == dato].index[0]
        elif (type(dato) == int):
            # Obtenemos el nombre de la pelicula
            title = df['title'][dato]
            posicion = dato

        idx = indices[posicion]   
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]  # Recomendamos las 5 películas más similares
        movie_indices = [i[0] for i in sim_scores]
        retorno = (f'Pelicula seleccionada: {title}, id: {posicion}\n\nPensamos que te puede interesar:\n{df['title'].iloc[movie_indices]}')
    
    return retorno 

# Prueba del modelo de recomendacion 
print(recomendar_peliculas_M3('fghfgh'))

La pelicula no existe


In [38]:
# Prueba del modelo de recomendacion 
M3_test01 = recomendar_peliculas_M3('Titanic')
print(M3_test01)

Pelicula seleccionada: Titanic, id: 1641

Pensamos que te puede interesar:
10247             A Touch of Spice
26091    Swinging with the Finkels
13251                  Girls Rock!
29486    Girltrash: All Night Long
9340             Undertaking Betty
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1265061261.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [39]:
M3_test02 = recomendar_peliculas_M3('Batman')
print(M3_test02)

Pelicula seleccionada: Batman, id: 585

Pensamos que te puede interesar:
36495          Pinocchio's Christmas
27719                      Pinocchio
803      The Adventures of Pinocchio
17967                      Pinocchio
30831            Pinocchio's Revenge
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1265061261.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [40]:
M3_test03 = recomendar_peliculas_M3('Sabrina')
print(M3_test03)

Pelicula seleccionada: Sabrina, id: 6

Pensamos que te puede interesar:
18011                    Margaret
21008    After School Midnighters
1364                       Mother
35246                 Zoolander 2
44857                Ugly Melanie
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1265061261.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [41]:
M3_test04 = recomendar_peliculas_M3('Toy Story')
print(M3_test04)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1265061261.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


Pelicula seleccionada: Toy Story, id: 0

Pensamos que te puede interesar:
15371                    Toy Story 3
3000                     Toy Story 2
24542                      Small Fry
10312         The 40 Year Old Virgin
23863    Andy Hardy's Blonde Trouble
Name: title, dtype: object


In [42]:
M3_test05 = recomendar_peliculas_M3('Pinocchio')
print(M3_test05)

Pelicula seleccionada: Pinocchio, id: 589

Pensamos que te puede interesar:
13844      Breakfast with Scot
1740              Nil by Mouth
8969         Across the Tracks
415      Beverly Hills Cop III
13951                Blue City
Name: title, dtype: object


C:\Users\Usuario\AppData\Local\Temp\ipykernel_12256\1265061261.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


Al agregarle la característica production_companies no se observó mejora en el modelo

---
Modelo de recomendación con las características overview, género, production_companies y director
---

In [87]:
# Caracteristicas combinadas
df['combined_features'] = df['genres'] + ' ' + df['overview'] + ' ' + df['production_companies'] + ' ' + df['director']

In [94]:
df['combined_features'][0]

"Animation Comedy Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences. Pixar Animation Studios John Lasseter"

In [89]:
# Vectorización
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

In [90]:
# Calcular similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [91]:
# Crear un índice para mapear los títulos a los índices, y nos aseguramos que no existan duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Robin Hood                     45449
Century of Birthing            45450
Betrayal                       45451
Satan Triumphant               45452
Queerama                       45453
Length: 44513, dtype: int64

In [92]:
# Función que realiza la recomendación
def recomendar_peliculas_M4(dato, cosine_sim = cosine_sim):
    if df[df['title'] == dato].empty:
        retorno = 'La pelicula no existe'
    
    else:
        if (type(dato) == str):
            # Obtengo la posicion
            title = dato
            posicion = df[df['title'] == dato].index[0]
        elif (type(dato) == int):
            # Obtenemos el nombre de la pelicula
            title = df['title'][dato]
            posicion = dato

        idx = indices[posicion]   
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]  # Recomendamos las 5 películas más similares
        movie_indices = [i[0] for i in sim_scores]
        retorno = (f'Pelicula seleccionada: {title}, id: {posicion}\n\nPensamos que te puede interesar:\n{df['title'].iloc[movie_indices]}')
    
    return retorno 

# Prueba del modelo de recomendacion 
print(recomendar_peliculas_M3('fghfgh'))

La pelicula no existe


In [93]:
M4_test01 = recomendar_peliculas_M3('Titanic')
print(M4_test01)
M4_test02 = recomendar_peliculas_M3('Batman')
print(M4_test02)
M4_test03 = recomendar_peliculas_M3('Sabrina')
print(M4_test03)
M4_test04 = recomendar_peliculas_M3('Toy Story')
print(M4_test04)
M4_test05 = recomendar_peliculas_M3('Pinocchio')
print(M4_test05)

Pelicula seleccionada: Titanic, id: 1641

Pensamos que te puede interesar:
10247             A Touch of Spice
26091    Swinging with the Finkels
13251                  Girls Rock!
29486    Girltrash: All Night Long
9340             Undertaking Betty
Name: title, dtype: object
Pelicula seleccionada: Batman, id: 585

Pensamos que te puede interesar:
36495          Pinocchio's Christmas
27719                      Pinocchio
803      The Adventures of Pinocchio
17967                      Pinocchio
30831            Pinocchio's Revenge
Name: title, dtype: object
Pelicula seleccionada: Sabrina, id: 6

Pensamos que te puede interesar:
18011                    Margaret
21008    After School Midnighters
1364                       Mother
35246                 Zoolander 2
44857                Ugly Melanie
Name: title, dtype: object
Pelicula seleccionada: Toy Story, id: 0

Pensamos que te puede interesar:
15371                    Toy Story 3
3000                     Toy Story 2
24542                

C:\Users\Usuario\AppData\Local\Temp\ipykernel_27636\1265061261.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


El modelo no mostró mejora en cuanto a la reomendacion de algunas películas como Batman y Piocchio.
Tal vez el orden de las columnas afecte en la recomendación 

---
Modelo de recomendación con las características overview, género, production_companies, actores y director
---

In [113]:
# se definen las caractersticas a analizar en una sola columna
df['combined_features'] =  df['overview'] + ' ' + df['genres'] + ' ' + df['production_companies'] + ' ' + df['production_countries'] + ' ' + df['director'] + ' ' + df['actor_names']
df['combined_features'][585]

"The Dark Knight of Gotham City begins his war on crime with his first major enemy being the clownishly homicidal Joker, who has seized control of Gotham's underworld. Fantasy Action PolyGram Filmed Entertainment The Guber-Peters Company Warner Bros. United Kingdom United States of America Tim Burton Jack Nicholson Michael Keaton Kim Basinger Michael Gough Pat Hingle Billy Dee Williams Jack Palance Jerry Hall Tracey Walter Lee Wallace William Hootkins Edwin Craig Robert Wuhl Vincent Wong Richard Strange Christopher Fairbank Richard Durden Sam Douglas Steve Plytas"

In [100]:
 # Vectorización
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['combined_features'])

# Calcular similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Crear un índice para mapear los títulos a los índices, y nos aseguramos que no existan duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices

title
Toy Story                          0
Jumanji                            1
Grumpier Old Men                   2
Waiting to Exhale                  3
Father of the Bride Part II        4
                               ...  
Robin Hood                     45449
Century of Birthing            45450
Betrayal                       45451
Satan Triumphant               45452
Queerama                       45453
Length: 44513, dtype: int64

In [109]:
# Función que realiza la recomendación
def recomendar_peliculas_M5(dato, cosine_sim = cosine_sim):
    if df[df['title'] == dato].empty:
        retorno = 'La pelicula no existe'
    
    else:
        if (type(dato) == str):
            # Obtengo la posicion
            title = dato
            posicion = df[df['title'] == dato].index[0]
        elif (type(dato) == int):
            # Obtenemos el nombre de la pelicula
            title = df['title'][dato]
            posicion = dato

        idx = indices[posicion]   
        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]  # Recomendamos las 5 películas más similares
        movie_indices = [i[0] for i in sim_scores]
        retorno = (f'\n\nPelicula seleccionada: {title}, id: {posicion}\nPensamos que te puede interesar:\n{df['title'].iloc[movie_indices]}\n\n')
    
    return retorno 

# Prueba del modelo de recomendacion 
print(recomendar_peliculas_M5('fghfgh'))

La pelicula no existe


In [110]:
# Prueba del modelo de recomendacion 
print(recomendar_peliculas_M5('fghfgh'))
M5_test01 = recomendar_peliculas_M3('Titanic')
print(M5_test01)
M5_test02 = recomendar_peliculas_M3('Batman')
print(M5_test02)
M5_test03 = recomendar_peliculas_M3('Sabrina')
print(M5_test03)
M5_test04 = recomendar_peliculas_M3('Toy Story')
print(M5_test04)
M5_test05 = recomendar_peliculas_M3('Pinocchio')
print(M5_test05)

Pelicula seleccionada: Titanic, id: 1641

Pensamos que te puede interesar:
10247             A Touch of Spice
26091    Swinging with the Finkels
13251                  Girls Rock!
29486    Girltrash: All Night Long
9340             Undertaking Betty
Name: title, dtype: object
Pelicula seleccionada: Batman, id: 585

Pensamos que te puede interesar:
36495          Pinocchio's Christmas
27719                      Pinocchio
803      The Adventures of Pinocchio
17967                      Pinocchio
30831            Pinocchio's Revenge
Name: title, dtype: object
Pelicula seleccionada: Sabrina, id: 6

Pensamos que te puede interesar:
18011                    Margaret
21008    After School Midnighters
1364                       Mother
35246                 Zoolander 2
44857                Ugly Melanie
Name: title, dtype: object
Pelicula seleccionada: Toy Story, id: 0

Pensamos que te puede interesar:
15371                    Toy Story 3
3000                     Toy Story 2
24542                

C:\Users\Usuario\AppData\Local\Temp\ipykernel_27636\1265061261.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  idx = indices[posicion]


In [23]:
# Librerias
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Carga el dataset de películas
df = pd.read_parquet('Data/dataset_movies_to_ML.parquet')


# Se convierten las columnas produccion_companies, production_companies, 'production_countries', 'director' y 'actor_names' en texto
# Convierte arrays de numpy a cadenas de texto por cada columna
df['genres'] = df['genres'].apply(lambda x: ' '.join(x))
df['production_companies'] = df['production_companies'].apply(lambda x: ' '.join(x))
df['production_countries'] = df['production_countries'].apply(lambda x: ' '.join(x))
df['director'] = df['director'].apply(lambda x: ' '.join(x))
df['actor_names'] = df['actor_names'].apply(lambda x: ' '.join(x))


# Se combinan las características a pasar al modelo
df['combined_features'] =  df['overview'] + ' ' + df['genres'] + ' ' + df['production_companies'] + ' ' + df['production_countries'] + ' ' + df['director'] + ' ' + df['actor_names']


# Vectorización
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Calcular similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Función que realiza la recomendación
def recomendar_peliculas_M6(dato, cosine_sim = cosine_sim):
    if len(df[df['title'] == dato]) == 0:
        retorno = 'La pelicula no existe'
    else:
        posicion = df[df['title'] == dato].index[0]
        print(type(posicion))
        sim_scores = list(enumerate(cosine_sim[posicion]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]  # Recomendamos las 5 películas similares
        movie_indices = [i[0] for i in sim_scores]
        retorno = (f'Pelicula seleccionada: {dato}, id: {posicion}\nPensamos que te puede interesar:\n{df['title'].iloc[movie_indices]}\n\n')

    return retorno 



# Prueba del modelo de recomendacion 
print(recomendar_peliculas_M6('fghfgh'))

M6_test01 = recomendar_peliculas_M6('Batman')
print(M6_test01)
# M6_test02 = recomendar_peliculas_M6('Batman')
# print(M6_test02)
# M6_test03 = recomendar_peliculas_M6('Star Wars')
# print(M6_test03)
# M6_test04 = recomendar_peliculas_M6('Toy Story')
# print(M6_test04)
# M6_test05 = recomendar_peliculas_M6('Pinocchio')
# print(M6_test05)





La pelicula no existe
<class 'numpy.int64'>


TypeError: 'list' object is not callable

In [22]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

def comprobar_pelicula(titulo):
    idx = -1
    for i, ind in enumerate (indices):
        if titulo == indices.index[i]:
            idx = i
            break
    return idx

bat = comprobar_pelicula('Sabrina')
bat

6

---
---
---

In [ ]:
# Vectorización
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['overview'])

# Calcular similitud del coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Crear un índice para mapear los títulos a los índices, y nos aseguramos que no existan duplicados
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [49]:
# Función que realiza la recomendación
def recomendar_peliculas_M10(dato, cosine_sim = cosine_sim):
    posicion = dato
    sim_scores = list(enumerate(cosine_sim[posicion]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Recomendamos las 5 películas similares
    movie_indices = [i[0] for i in sim_scores]
    retorno = (f'id Pelicula seleccionada: {dato}\nPensamos que te puede interesar:\n{df['title'].iloc[movie_indices]}\n\n')
    return retorno 

In [50]:
# Función para comprobar existencia de peliculas, retorna el indice de la primera que encuentre
def comprobar_pelicula(titulo):
    idx = -1
    for i, ind in enumerate (indices):
        if titulo == indices.index[i]:
            idx = i
            break
    if idx < 0:
        retorno = 'La pelicula no existe en la DB'
    else:
        retorno = recomendar_peliculas_M10(idx)
    return retorno

In [51]:
# Prueba del modelo de recomendacion 
print(comprobar_pelicula('fghfgh'))
M10_test01 = comprobar_pelicula('Titanic')
print(M10_test01)
M10_test02 = comprobar_pelicula('Batman')
print(M10_test02)
M10_test03 = comprobar_pelicula('Sabrina')
print(M10_test03)
M10_test04 = comprobar_pelicula('Toy Story')
print(M10_test04)
M10_test05 = comprobar_pelicula('Pinocchio')
print(M10_test05)

La pelicula no existe en la DB
Pelicula seleccionada: 1625, id: 1625
Pensamos que te puede interesar:
3287      Raise the Titanic
26985    Sailor of the King
38679          Dustbin Baby
2579     The Legend of 1900
3288                Titanic
Name: title, dtype: object


Pelicula seleccionada: 583, id: 583
Pensamos que te puede interesar:
9239     Batman Beyond: Return of the Joker
18276                 The Dark Knight Rises
25286                     Batman vs Dracula
1493                         Batman & Robin
15534            Batman: Under the Red Hood
Name: title, dtype: object


Pelicula seleccionada: 6, id: 6
Pensamos que te puede interesar:
18011                    Margaret
21008    After School Midnighters
1364                       Mother
35246                 Zoolander 2
44857                Ugly Melanie
Name: title, dtype: object


Pelicula seleccionada: 0, id: 0
Pensamos que te puede interesar:
15371                    Toy Story 3
3000                     Toy Story 2
24542   

Mediante la implementación de la función que comprueba la existencia de la película pasada por parametro mediante su titulo y la indexación correcta, se puede observar que la recomendación sólo de la característica 'overview' es capaz de recomendar películas con demasiada similitud, tras a ello, se elige el modelo por columna 'overview' como la caracteristica de entrenamiento para el modelo, mediante ello se realiza la recomendación.